In [10]:
#bibliotecas de análise
import pandas as pd
import numpy as np

#pacotes para gráficos
import graphviz
import matplotlib.pyplot as plt

#pacote para clustering
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

#pacote para realização do PCA
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler

#pacote para arvore de decião
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

#pacotes para o randon florest
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

Carregando a tabela de dados préprocessada

In [4]:
df = pd.read_csv('../Scripts/df_metal_ML.csv', sep = ',', header = 0)

In [6]:
df.describe(include = 'all')

,Material,Su,Sy,A5,Bhn,E,G,mu,Ro,categoria
count,675,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000
unique,630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,GOST Steel 50 GOST 1577-93 normalized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,584.880000,409.570370,14.061333,101.502222,150456.296296,108336.296296,0.310726,5703.684444,3.048889
std,NaN,389.751772,339.980396,11.094605,124.457343,65143.432944,186578.885476,0.016529,2610.918867,1.380979
min,NaN,69.000000,28.000000,0.000000,0.000000,73000.000000,26000.000000,0.290000,1750.000000,1.000000
25%,NaN,276.000000,200.000000,6.000000,0.000000,73000.000000,26000.000000,0.300000,2700.000000,1.000000
50%,NaN,524.000000,320.000000,12.000000,60.000000,200000.000000,77000.000000,0.300000,7820.000000,3.000000
75%,NaN,733.000000,466.500000,20.000000,176.500000,207000.000000,80000.000000,0.330000,7860.000000,4.000000


In [88]:
var = df.drop(['Material', 'categoria'], axis = 1)
df_resposta = df['categoria'].values

In [89]:
df_resposta.shape

(675,)

In [90]:
np.unique(df_resposta, return_counts = True)

(array([1, 2, 3, 4, 5]), array([177,  14, 165, 237,  82]))

In [91]:
#padronização dos dados
scale_obj = StandardScaler()
df_var = scale_obj.fit_transform(var.astype(float))

In [108]:
#cross validation
from sklearn.model_selection import train_test_split

treino_var, teste_var, treino_resposta, teste_resposta = train_test_split(df_var, df_resposta, test_size=0.3)

PCA

In [128]:
pca = PCA(n_components=2)
treino_var_pca = pca.fit(treino_var)
r2 = pca.transform(treino_var)

In [129]:

print(f'Variacia expelicada pelo PCA: {pca.explained_variance_ratio_}')

Variacia expelicada pelo PCA: [0.54735342 0.20616822]


In [135]:
def cores (valores):
    cores ={key:index for index, key in enumerate(valores.unique())}
    return valores.apply(lambda x: cores[x])

In [136]:
plt.scatter(r2[:,0], r2[:, 1], c = cores(df_resposta))

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

In [ ]:
df_pc = pd.DataFrame(data = treino_var, )Í

In [ ]:
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Componente principal 1', fontsize = 15)
ax.set_ylabel('Componente principal 2', fontsize = 15)